# YASA

This notebook demonstrates how to use YASA to perform **multi-channels sleep spindles detection** from a NumPy array (example 1) or a MNE Raw object (example 2).

Please make sure to install YASA first by typing the following line in your terminal or command prompt:

`pip install --upgrade yasa`

**Important**
- The data must be a numpy array of shape *(n_channels, n_samples)*.
- The sampling frequency `sf` must be the same for all channels.
- A list of the channel names (`ch_names`) must be provided as well.
- The unit of the data must be $\mu V$. Note that the default unit in [MNE](https://martinos.org/mne/dev/generated/mne.io.Raw.html) is $V$. Therefore, if you use MNE, you must multiply your data by 1e6 (1 $V$ = 1,000,000 $\mu V$).

## Example 1: Using NumPy

To illustrate the multi-channel spindles detection, we load a full-night 3-channels dataset (Cz, Fz, Pz) sampled at 100 Hz. The data is in compressed NumPy format (*.npz*).

In [1]:
import yasa
import numpy as np

# Load data
f = np.load('data_full_6hrs_100Hz_Cz+Fz+Pz.npz')
data, chan = f['data'], f['chan']
sf = 100.
times = np.arange(data.size) / sf

print(data.shape, chan)
print(np.round(data[:, 0:5], 3))

(3, 2161058) ['Cz' 'Fz' 'Pz']
[[15.797 22.307 39.922 25.657 27.094]
 [16.896 26.385 40.966 21.833 24.456]
 [ 5.899 14.297 36.592 26.094 23.395]]


*************

**Applying the detection**

To apply the multi-channel detection, we use the `yasa.spindles_detect_multi` function. Read the doc below:

In [2]:
# Print the documentation
yasa.spindles_detect_multi?

Signature: yasa.spindles_detect_multi(data, sf, ch_names, multi_only=False, **kwargs)
Docstring:
Multi-channel spindles detection.

Parameters
----------
data : array_like
    Multi-channel data. Unit must be uV and shape (n_chan, n_samples).
    If you used MNE to load the data, you should pass `raw._data * 1e6`.
sf : float
    Sampling frequency of the data in Hz.
    If you used MNE to load the data, you should pass `raw.info['sfreq']`.
ch_names : list of str
    Channel names.
    If you used MNE to load the data, you should pass `raw.ch_names`.
multi_only : boolean
    Define the behavior of the multi-channel detection. If True, only
    spindles that are present on at least two channels are kept. If False,
    no selection is applied and the output is just a concatenation of the
    single-channel detection dataframe. Default is False.
**kwargs
    Keywords arguments that are passed to the `spindles_detect` function.

Returns
-------
sp_params : pd.DataFrame
    Pandas DataFrame:

In [3]:
sp = yasa.spindles_detect_multi(data, sf, ch_names=chan, multi_only=False)
print(sp.shape[0], 'spindles detected.')
sp.head().round(3)

2205 spindles detected.


,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
0,522.54,523.79,1.25,45.509,10.850,2.131,0.470,13.083,16.0,0.190,Cz,0
1,585.51,586.38,0.87,59.435,13.185,2.255,0.473,12.946,11.0,0.659,Cz,0
2,598.06,599.54,1.48,79.922,15.977,2.373,0.421,12.889,19.0,0.376,Cz,0
3,604.36,605.12,0.76,60.649,12.520,2.209,0.350,12.646,9.0,0.481,Cz,0
4,607.53,608.05,0.52,50.211,13.847,2.358,0.258,13.452,6.0,0.849,Cz,0


In [4]:
# We print the number of spindles detected per channel, as well as the mean spindles properties per channel.
display(sp['Channel'].value_counts())
display(sp.groupby('Channel').mean().round(2))

Pz    786
Cz    764
Fz    655
Name: Channel, dtype: int64

,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,IdxChannel
Channel,,,,,,,,,,,
Cz,12389.39,12390.36,0.97,76.19,16.80,2.42,0.39,12.67,11.96,0.51,0
Fz,12609.81,12610.71,0.90,67.54,14.97,2.31,0.37,12.50,11.00,0.51,1
Pz,12405.95,12406.92,0.97,68.77,15.09,2.32,0.38,12.68,12.00,0.51,2


In [5]:
# For plotting purposes, we can easily extract a boolean vector that has the same size as the data
bool_vector = yasa.get_bool_vector(data, sf, sp)
print(bool_vector.sum(1))
print(bool_vector)

[74835 59843 77198]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Find spindles that are present on at least two channels

Using the `multi_only=True` argument, we force the detection to keep only the spindles that are present on at least two different channels. Spindles are considered the same if their start or end times fall within the same second. In other words, start and end times are rounded to the nearest integer and compared across channels.

In [6]:
sp_multi = yasa.spindles_detect_multi(data, sf, ch_names=chan, multi_only=True, remove_outliers=True)
print(sp_multi.shape[0], 'spindles detected that are common to at least two electrodes.')
sp_multi.head().round(3)

1519 spindles detected that are common to at least two electrodes.


,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
0,585.51,586.38,0.87,59.435,13.185,2.255,0.473,12.946,11.0,0.659,Cz,0
1,598.06,599.54,1.48,79.922,15.977,2.373,0.421,12.889,19.0,0.376,Cz,0
2,604.36,605.12,0.76,60.649,12.520,2.209,0.350,12.646,9.0,0.481,Cz,0
3,655.06,655.88,0.82,47.596,10.832,2.149,0.369,13.297,11.0,0.530,Cz,0
4,725.60,726.88,1.28,77.591,18.836,2.370,0.288,12.370,15.0,0.698,Cz,0


In [7]:
display(sp_multi['Channel'].value_counts())
display(sp_multi.groupby('Channel').mean().round(2))

Cz    579
Pz    500
Fz    440
Name: Channel, dtype: int64

,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,IdxChannel
Channel,,,,,,,,,,,
Cz,12584.65,12585.61,0.96,74.25,16.34,2.41,0.39,12.63,11.80,0.52,0
Fz,13090.57,13091.43,0.86,65.38,14.53,2.30,0.37,12.50,10.44,0.51,1
Pz,12496.29,12497.27,0.98,67.65,14.85,2.32,0.39,12.67,12.18,0.51,2


### Find spindles that are present on ALL channels only

The code below show how to apply an even more stringent selection: only the spindles that are common across all electrodes are kept. This method is not natively implemented in YASA.

In [8]:
from functools import reduce
grp_start = sp.groupby('Channel')['Start'].apply(lambda x: list(np.round(x).astype(int))).to_dict()
grp_end = sp.groupby('Channel')['End'].apply(lambda x: list(np.round(x).astype(int))).to_dict()

intersect_start = reduce(np.intersect1d, (grp_start[c] for c in sp['Channel'].unique()))
intersect_end = reduce(np.intersect1d, (grp_end[c] for c in sp['Channel'].unique()))

idx_start = np.in1d(sp['Start'].round().astype(int), intersect_start)
idx_end = np.in1d(sp['End'].round().astype(int), intersect_end)
idx_good = np.logical_or(idx_start, idx_end)

# Now we keep only these spindles in the dataframe
print(sp[idx_good].shape[0], 'unique spindles that are common across ALL channels.')
sp[idx_good].head().round(3)

1219 unique spindles that are common across ALL channels.


,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
3,604.36,605.12,0.76,60.649,12.520,2.209,0.350,12.646,9.0,0.481,Cz,0
6,631.57,632.18,0.61,158.228,37.444,3.150,0.436,12.612,7.0,0.387,Cz,0
7,655.06,655.88,0.82,47.596,10.832,2.149,0.369,13.297,11.0,0.530,Cz,0
9,735.12,736.39,1.27,80.165,16.962,2.543,0.579,12.675,15.0,0.609,Cz,0
11,744.01,745.47,1.46,79.723,16.769,2.602,0.636,12.773,19.0,0.660,Cz,0


### Test with a bad channel

Here, we create a fake channel with no spindle to test the detection.

In [9]:
# data[1, :] = np.random.rand(data.shape[1])
data[1, :] = np.sin(0.1 * np.arange(data.shape[1]))
sp_bad = yasa.spindles_detect_multi(data, sf, ch_names=chan, multi_only=False)
display(sp_bad['Channel'].value_counts())
sp.head().round(3)

07-Feb-19 15:50:24 | WARNING | No spindles were found in data. Returning None.
07-Feb-19 15:50:24 | WARNING | No spindles were found in channel Fz.


Pz    786
Cz    764
Name: Channel, dtype: int64

,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
0,522.54,523.79,1.25,45.509,10.850,2.131,0.470,13.083,16.0,0.190,Cz,0
1,585.51,586.38,0.87,59.435,13.185,2.255,0.473,12.946,11.0,0.659,Cz,0
2,598.06,599.54,1.48,79.922,15.977,2.373,0.421,12.889,19.0,0.376,Cz,0
3,604.36,605.12,0.76,60.649,12.520,2.209,0.350,12.646,9.0,0.481,Cz,0
4,607.53,608.05,0.52,50.211,13.847,2.358,0.258,13.452,6.0,0.849,Cz,0


In [10]:
bool_vector = yasa.get_bool_vector(data, sf, sp_bad)
bool_vector.sum(1)

array([74835,     0, 77198])

## Example 2: Using a Raw object from MNE-Python

This example demonstrates how to manipulate [MNE Raw object](https://mne-tools.github.io/stable/generated/mne.io.Raw.html#mne.io.Raw). The MNE package has several [functions](https://mne-tools.github.io/stable/python_reference.html#module-mne.io) to load the most standard EEG file formats (EDF, BrainVision, EEGLab, FieldTrip...).

### Load using MNE
For the sake of this example, we'll load a PSG file encoded in the native MNE format (*.fif) using the `mne.io.read_raw_fif` function.

In [11]:
import mne

# Load the raw object
raw = mne.io.read_raw_fif('sub-02_mne_raw.fif', preload=True)

Opening raw data file C:\Users\Raphael\Desktop\yasa\notebooks\sub-02_mne_raw.fif...
    Range : 0 ... 293999 =      0.000 ...  2939.990 secs
Ready.
Reading 0 ... 293999  =      0.000 ...  2939.990 secs...


In [12]:
# Let's have a look at the data
print('Chan =', raw.ch_names)
print('Sampling frequency =', raw.info['sfreq'])
print('Data shape =', raw._data.shape)

Chan = ['F3', 'F4', 'C3', 'C4', 'O1', 'O2', 'EOG1', 'EOG2', 'EMG1']
Sampling frequency = 100.0
Data shape = (9, 294000)


### Applying YASA

Before we can apply the detection on our Raw object, there are *two important things that we need to do*:

1. We need to select only the channels of interests. Indeed, we do not want to run the detection on EMG, EOG or ECG channels.
2. We need to convert the data amplitude to micro-volts ($\mu V$). The default unit in MNE is volts, which means that we need to scale our data by a factor of 1,000,000 (= 1e6).

#### Channel selection

In [13]:
# 1 - Select only the EEG channels
raw_eeg = raw.copy().pick_types(eeg=True)
print('Chan =', raw_eeg.ch_names)

Chan = ['F3', 'F4', 'C3', 'C4', 'O1', 'O2']


In [14]:
# 1bis - Remove O1 and O2 channels from data
raw_eeg.drop_channels(['O1', 'O2'])
print('Chan =', raw_eeg.ch_names)

Chan = ['F3', 'F4', 'C3', 'C4']


#### Data scaling

Are we absolutely sure that the data are in volts and not micro-volts? To quickly check that, let's display the maximum value of each channel:

In [15]:
# Maximum value of each channel
raw_eeg._data.max(axis=1)

array([0.00116572, 0.00108572, 0.00053502, 0.00056241])

As you can see above, the data are definitely in volts and not micro-volts (otherwise we would have expected values that at the very least superior to 10). We therefore need to convert our data to micro-volts (1 $V$ = 1,000,000 $\mu V$):

In [16]:
# Convert V --> uV
data_mv = raw_eeg._data * 1e6

#### Run the detection

In [17]:
sp = yasa.spindles_detect_multi(data=data_mv, 
                                sf=raw_eeg.info['sfreq'], 
                                ch_names=raw_eeg.ch_names)
sp.head().round(3)

,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
0,879.76,880.53,0.77,49.915,9.490,2.017,0.393,13.803,10.0,0.359,F3,0
1,1041.76,1042.24,0.48,34.017,8.702,1.779,0.328,13.075,6.0,0.286,F3,0
2,1068.45,1069.46,1.01,39.570,7.860,1.521,0.268,13.472,13.0,0.696,F3,0
3,1078.82,1079.38,0.56,41.574,9.711,2.070,0.409,13.175,8.0,0.684,F3,0
4,1085.62,1086.04,0.42,22.991,5.182,1.615,0.350,13.981,5.0,0.186,F3,0
